# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 10:34AM,243870,10,MSP213012,"Methapharm, Inc.",Released
1,Aug 10 2022 10:33AM,243869,10,MSP213006,"Methapharm, Inc.",Released
2,Aug 10 2022 10:33AM,243869,10,MSP213007,"Methapharm, Inc.",Released
3,Aug 10 2022 10:33AM,243869,10,MSP213009,"Methapharm, Inc.",Released
4,Aug 10 2022 10:33AM,243869,10,MSP213010,"Methapharm, Inc.",Released
5,Aug 10 2022 10:33AM,243869,10,MSP213011,"Methapharm, Inc.",Released
6,Aug 10 2022 10:22AM,243867,12,8385111,"NBTY Global, Inc.",Executing
7,Aug 10 2022 10:22AM,243864,15,8381355,"Mizner Bioscience, LLC",Released
8,Aug 10 2022 10:22AM,243864,15,8381356,"Mizner Bioscience, LLC",Released
9,Aug 10 2022 10:22AM,243864,15,8381357,"Mizner Bioscience, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,243865,Released,1
39,243866,Released,20
40,243867,Executing,1
41,243869,Released,5
42,243870,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243865,NaN,NaN,1.0
243866,NaN,NaN,20.0
243867,NaN,1.0,NaN
243869,NaN,NaN,5.0
243870,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243768,23.0,2.0,0.0
243769,0.0,0.0,9.0
243771,1.0,4.0,1.0
243778,16.0,12.0,1.0
243783,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243768,23,2,0
243769,0,0,9
243771,1,4,1
243778,16,12,1
243783,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243768,23,2,0
1,243769,0,0,9
2,243771,1,4,1
3,243778,16,12,1
4,243783,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243768,23,2,
1,243769,,,9
2,243771,1,4,1
3,243778,16,12,1
4,243783,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc."
1,Aug 10 2022 10:33AM,243869,10,"Methapharm, Inc."
6,Aug 10 2022 10:22AM,243867,12,"NBTY Global, Inc."
7,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC"
10,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.
30,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC"
35,Aug 10 2022 10:19AM,243865,16,American International Chemical
36,Aug 10 2022 10:19AM,243847,20,Alumier Labs Inc.
56,Aug 10 2022 10:08AM,243862,15,"Brookfield Pharmaceuticals, LLC"
77,Aug 10 2022 10:06AM,243860,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc.",,,1
1,Aug 10 2022 10:33AM,243869,10,"Methapharm, Inc.",,,5
2,Aug 10 2022 10:22AM,243867,12,"NBTY Global, Inc.",,1,
3,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",,,3
4,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,,,20
5,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC",,,5
6,Aug 10 2022 10:19AM,243865,16,American International Chemical,,,1
7,Aug 10 2022 10:19AM,243847,20,Alumier Labs Inc.,,,20
8,Aug 10 2022 10:08AM,243862,15,"Brookfield Pharmaceuticals, LLC",,2,19
9,Aug 10 2022 10:06AM,243860,15,"Alliance Pharma, Inc.",,,36


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc.",1,,
1,Aug 10 2022 10:33AM,243869,10,"Methapharm, Inc.",5,,
2,Aug 10 2022 10:22AM,243867,12,"NBTY Global, Inc.",,1,
3,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",3,,
4,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,20,,
5,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC",5,,
6,Aug 10 2022 10:19AM,243865,16,American International Chemical,1,,
7,Aug 10 2022 10:19AM,243847,20,Alumier Labs Inc.,20,,
8,Aug 10 2022 10:08AM,243862,15,"Brookfield Pharmaceuticals, LLC",19,2,
9,Aug 10 2022 10:06AM,243860,15,"Alliance Pharma, Inc.",36,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc.",1,,
1,Aug 10 2022 10:33AM,243869,10,"Methapharm, Inc.",5,,
2,Aug 10 2022 10:22AM,243867,12,"NBTY Global, Inc.",,1,
3,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",3,,
4,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Aug 10 2022 10:33AM,243869,10,"Methapharm, Inc.",5.0,NaN,NaN
2,Aug 10 2022 10:22AM,243867,12,"NBTY Global, Inc.",NaN,1.0,NaN
3,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",3.0,NaN,NaN
4,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Aug 10 2022 10:33AM,243869,10,"Methapharm, Inc.",5.0,0.0,0.0
2,Aug 10 2022 10:22AM,243867,12,"NBTY Global, Inc.",0.0,1.0,0.0
3,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",3.0,0.0,0.0
4,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2194513,45.0,12.0,9.0
102,243818,1.0,0.0,0.0
12,731500,0.0,3.0,0.0
15,975449,63.0,2.0,0.0
16,243865,1.0,0.0,0.0
19,975257,7.0,24.0,31.0
20,1462864,66.0,30.0,33.0
21,487658,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2194513,45.0,12.0,9.0
1,102,243818,1.0,0.0,0.0
2,12,731500,0.0,3.0,0.0
3,15,975449,63.0,2.0,0.0
4,16,243865,1.0,0.0,0.0
5,19,975257,7.0,24.0,31.0
6,20,1462864,66.0,30.0,33.0
7,21,487658,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,45.0,12.0,9.0
1,102,1.0,0.0,0.0
2,12,0.0,3.0,0.0
3,15,63.0,2.0,0.0
4,16,1.0,0.0,0.0
5,19,7.0,24.0,31.0
6,20,66.0,30.0,33.0
7,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,45.0
1,102,Released,1.0
2,12,Released,0.0
3,15,Released,63.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21
Status,,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,31.0,33.0,0.0
Executing,12.0,0.0,3.0,2.0,0.0,24.0,30.0,0.0
Released,45.0,1.0,0.0,63.0,1.0,7.0,66.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,31.0,33.0,0.0
1,Executing,12.0,0.0,3.0,2.0,0.0,24.0,30.0,0.0
2,Released,45.0,1.0,0.0,63.0,1.0,7.0,66.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,31.0,33.0,0.0
1,Executing,12.0,0.0,3.0,2.0,0.0,24.0,30.0,0.0
2,Released,45.0,1.0,0.0,63.0,1.0,7.0,66.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()